Notebook settings

In [1]:
%pylab inline
np.set_printoptions(linewidth=120)
plt.rcParams["figure.figsize"] = (18,10)
plt.rcParams["legend.loc"] = "lower right"

Populating the interactive namespace from numpy and matplotlib


Set the path to the key file and change working-directory to the root directory of the dataprocessing framework

In [2]:
import os       
os.environ['EXOT_ACCESS_DIR'] = "/local/scratch/toolkit/benchmark_platforms_access/"
os.chdir('/local/scratch/toolkit/datapro/')
os.getcwd()

'/local/scratch/toolkit/datapro'

# Imports

In [3]:
import toml    as TOML              # TOML library to handle config files
import pandas  as pd                # Pandas for data manipulation
import seaborn as sb                # Statistical data visualisation
import pathlib as PATH              # Used to convert strings to path objects for (de-)serialisation
import types   as tp                # Library used for online method development

#from copy   import copy, deepcopy   # TODO
#from pprint import pprint           # TODO

# scikit-learn packets
import sklearn.base                 #
import sklearn.svm                  # LinearSVC, SVC
import sklearn.pipeline             # make_pipeline
import sklearn.preprocessing        # StandardScaler
import sklearn.decomposition        # PCA
import sklearn.naive_bayes          # GaussianNB
import sklearn.metrics              # CM

# DLE-Suite packets
import exot                       # Dataprocessing
import exot.experiment            # Experiment for execution
import exot.util                  # General utilities
import exot.layer                 # Data manipulation layers
from exot.util       import *
from exot.layer      import *
from exot.channel    import *
from exot.experiment import *

# Generate Experiment
#### 1. Load config

In [4]:
config = TOML.load("./configurations/examples/thermal-cc_performance.toml")
print(TOML.dumps(config))

name = "Test_Thermal"
save_path = "./data"
backup_path = "./data/_backup"
experiment_exists_action = "move"

[EXPERIMENT]
type = "PerformanceExperiment"
channel = "Thermal"

[EXPERIMENT.GENERAL]
latency = 10
fan = true
governors = "userspace"
frequencies = "max"
sampling_period = 0.001

[EXPERIMENT.PHASES.train]
bit_count = 60
symbol_rates = [ 1,]
repetitions = 1

[EXPERIMENT.LAYERS.src]
name = "SourcePassthrough"

[EXPERIMENT.LAYERS.lne]
name = "ManchesterLineCoding"

[EXPERIMENT.LAYERS.rdp]
name = "CoreActivation"

[EXPERIMENT.LAYERS.io]
name = "TimeValue"

[EXPERIMENT.LAYERS.src.params]

[EXPERIMENT.LAYERS.lne.params]

[EXPERIMENT.LAYERS.rdp.params]
saturating = true

[EXPERIMENT.LAYERS.io.params]
output_timing_multiplier = 1000000000.0
input_timing_multiplier = 1000000000.0

[ENVIRONMENTS.Radagast.APPS.snk]
executable = "meter_thermal_msr"
zone = "host"

[ENVIRONMENTS.Radagast.APPS.src]
executable = "generator_utilisation_mt"
zone = "host"

[ENVIRONMENTS.Radagast.src.generator]
cor

#### 2. Instantiate an experiment

Experiment factory will take the type and channel from the config.

In [5]:
channel    = ChannelFactory()(config["EXPERIMENT"]["channel"])
experiment = ExperimentFactory()(config["EXPERIMENT"]["type"], config=config, channel=channel)

2019-11-18 10:53:30 :: bootstrapped layer 'src' with SourcePassthrough
2019-11-18 10:53:30 :: bootstrapped layer 'lne' with ManchesterLineCoding
2019-11-18 10:53:30 :: bootstrapped layer 'rdp' with CoreActivation
2019-11-18 10:53:30 :: bootstrapped layer 'io' with TimeValue


In [6]:
experiment.generate()
#experiment.print_duration()

#### 3a. Eyeballing

You can access the streams via specific properties.

In [7]:
phase = "train"
run_id = 0
run = experiment.phases[phase][run_id]

# Access output streams
run.o_streams #-> mapping
#run.o_{bit,sym,lne,rdp,raw}stream
#print("-------------------------------- Bitstream -------------------------------------------------------")
#print(run.o_bitstream)
#print("-------------------------------- Symstream -------------------------------------------------------")
#print(run.o_symstream)
#print("-------------------------------- LNEstream -------------------------------------------------------")
#print(run.o_lnestream)
#print("-------------------------------- RDPstream -------------------------------------------------------")
#print(run.o_rdpstream)
#print("-------------------------------- Rawstream -------------------------------------------------------")
#print(run.o_rawstream)

{'bitstream': array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]),
 'symstream': array([1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]),
 'lnestream': array([ 0, -1, -1,  0, -1,  0,  0, -1,  0, -1, -1,  0, -1,  0,  0, -1,  0, -1,  0, -1, -1,  0,  0, -1, -1,  0, -1,  0,
         0, -1, -1,  0,  0, -1,  0, -1, -1,  0,  0, -1, -1,  0, -1,  0,  0, -1,  0, -1,  0, -1, -1,  0, -1,  0,  0, -1,
        -1,  0, -1,  0,  0, -1, -1,  0,  0, -1, -1,  0,  0, -1, -1,  0,  0, -1, -1,  0,  0, -1, -1,  0, -1,  0, -1,  0,
         0, -1, -1,  0, -1,  0,  0, -1, -1,  0,  0, -1,  0, -1,  0, -1,  0, -1,  0, -1, -1,  0,  0, -1, -1,  0, -1,  0,
        -1,  0,  0, -1, -1,  0,  0, -1]),
 'rdpstream':      timestamp  t440p
 0     

#### 4. Write

Writes to disk, serialises the experiment, runs, and all streams.

In [8]:
experiment.write()

In [9]:
experiment.print_duration()

2019-11-18 10:53:30 :: Estimated duration of a single repetition of the train phase is 0:01:12
2019-11-18 10:53:30 :: This results in a total estimated duration of 0:01:12 for all repetitions.


## Execute

Running `execute` will execute in all environments present in the configuration.

In [10]:
#print("Available environments are: " + str(experiment.available_environments))
#experiment.execute_in_environment("Gandalf", ["train"])
experiment.execute_in_environment("Radagast", ["train"])

2019-11-18 10:53:30 :: executing phases: ['train'] in env 'Radagast'
2019-11-18 10:53:32 :: Radagast->host: connected: SSHUnixDriver
2019-11-18 10:53:32 :: Radagast->host: sending experiment
2019-11-18 10:53:33 :: Radagast->host: experiment sent!
2019-11-18 10:53:33 :: Radagast->host: configuring to {'latency': 10, 'fan': True, 'governors': 'userspace', 'frequencies': 'max', 'sampling_period': 0.001}
2019-11-18 10:53:40 :: executing in env: 'Radagast', resume: False, reps: [0], run: <Run at 0x7f9fc820d940 (digested, config={'phase': 'train', 'bit_count': 60, 'symbol_rate': 1, 'symbol_rate_id': 0, 'repetitions': 1})>
2019-11-18 10:53:40 :: executing in env: Radagast, rep: 0, estimated duration: 60.0s, run: <Run at 0x7f9fc820d940 (digested, config={'phase': 'train', 'bit_count': 60, 'symbol_rate': 1, 'symbol_rate_id': 0, 'repetitions': 1})>
2019-11-18 10:54:52 :: Radagast->host: experiment logs fetched!
2019-11-18 10:54:52 :: Radagast->host: cleaning up
2019-11-18 10:54:59 :: Radagast->h

## Analyse

TODO: In the *analysis.local.ipynb* file in my workspace you can find a more complex example of an analysis.

#### 0. (Optional) Load from serialised experiment

Useful when you restarted the kernel, etc. Be aware that if you have the "move" option set up as the *experiment_exists_action* in the config, the older one will be timestamped and moved. To read it back, you can provide `replace=False` which handles the case where there are inconsistencies between the serialised experiment and the config files (primarily the experiment name, due to the timestamping). We discussed that quite a while ago, the result of the discussion was that the experiment should update itself with the config already present.

In [11]:
raise Exception()

Exception: 

In [ ]:
# path_to_serialised_experiment = PATH.Path("data/XXX/_experiment.pickle")
#path_to_serialised_experiment = "/local/scratch/dle/datapro/data/Haswell_Thermal_2019-05-24_16-23-01/_experiment.pickle"
#path_to_serialised_experiment = "/local/scratch/dle/datapro/data/Haswell_Thermal/_experiment.pickle"
#path_to_serialised_experiment = PATH.Path("data/Haswell_Thermal/_experiment.pickle")
path_to_serialised_experiment = PATH.Path("data/Test_Thermal/_experiment.pickle")
experiment = PerformanceExperiment.read(path_to_serialised_experiment, replace=False)

#### 1. Eyeballing single runs to check if experiment went well
##### 1a. Choose a run

In [ ]:
phase = "train"
symbol_rate = 1
run = experiment.phases[phase][symbol_rate]; run

##### 1a. Get available environments

In [ ]:
run.parent.layers.io.get_available_environments()

##### 1b. Get raw measurements

- The `get_measurements` function requires the I/O layer to have at least the environment, and the repetition number configured.
- `describe_measurements` will tell you what columns are available, with more detail.

In [ ]:
run.parent.layers.io.config.update(env="Radagast", rep=0)
measurements = run.parent.layers.io.get_measurements()
print(run.parent.layers.io.describe_measurements(measurements))

The raw measurements are packed into a pandas DataFrame and can be used for eyeballing the data.

##### 1.c Ingest the run

To ingest the run, and pipe the measurements through the layers, you need to provide the necessary runtime configuration. Most of it is passed by default, with the notable exception of the *Matcher*.

The *Matcher* defines which columns will be taken from the raw measurements and passed on to the layers. For the thermal channel, you'd use:

```python
env = "Gandalf"
rep = 0
core = 4
matcher = wrangle.Matcher("thermal", "msr", ["core"], [core])
ingest_args = dict(io={"env": env, "rep": rep, "matcher": matcher})
```

You can also provide other parameters, e.g. the *decision device*. For eval runs, you'd have to get it from a train run, via `run.parent.layers.lne.decision_device` property.

In [ ]:
ingest_args = dict(
  lne={"decision_device": sklearn.pipeline.make_pipeline(sklearn.preprocessing.StandardScaler(), 
                                                         sklearn.svm.LinearSVC())},
  io={"env": 'Radagast', "rep": 0, "matcher": datapro.util.wrangle.Matcher("thermal", "msr", ["core"], [2])}
#  io={"env": 'Radagast', "rep": 0, "matcher": datapro.util.wrangle.Matcher("thermal", "msr", ["core"], list(range(0,1)))}
)
run.ingest(**ingest_args)

After running ingest, the run's input streams will be populated, and can be accessed via:

- `run.i_bitstream`
- `run.i_symstream`
- `run.i_lnestream`
- `run.i_rdpstream`
- `run.i_rawstream`

##### 1d. Plot raw measurements

In [ ]:
run.plot_rawstream()

##### 1f. Plot sliced trace

In [ ]:
run.plot_slicing()

#### 2. Analyse the whole experiment and get the final results.

#### 3. Develop new functionality or modify functionality online

In order to allow more interactive development of the data analysis flow, methods can be changed online before they are pushed to the code-base. In the example below, the plot_rawstream method is updated, specifying different colours.

Be aware that in this example, the plot_slicing method is only updated for this one particular run object. However, as the layers are owned by the experiment, changing an encode or decode method using this flow, affects all runs.

In [ ]:
from matplotlib import cm

def plot_rawstream(self):
    self.i_rawstream.plot(
                           0, 
                           range(1,self.i_rawstream.shape[1]), 
                           subplots=True, 
                           cmap=cm.get_cmap('Spectral'), 
                           title="New rawstream plot"
                         )

#-------------------------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------------
run.plot_rawstream()
run.plot_rawstream = tp.MethodType(plot_rawstream, run)
run.plot_rawstream()